# Silver layer

This notebook orchestrates the Silver layer processing.

Silver validations are table-specific, so each table has its own notebook. This driver notebook executes the correct notebook based on parameters and writes execution logs to the control.logs Delta table.

Parameters used by the pipeline to control execution:

* **in_parameter_run_id**: unique identifier for the pipeline run
* **in_parameter_process_date**: execution date for lineage
* **in_parameter_table_name**: which table to process
* **in_parameter_is_fact_table**: controls whether year is required (fact table)
* **in_parameter_year**: partition to process (fact tables only)
* **out_parameter_count_processed**: output parameter

In [ ]:
in_parameter_run_id = 0
in_parameter_process_date = ""
in_parameter_table_name = ""
in_parameter_is_fact_table = True
in_parameter_year = 2010
out_parameter_count_processed = 0

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

## 1. Initialize execution logs

Create an in-memory log structure to capture success/failure for the notebook execution before persisting it to the Lakehouse control table.

In [ ]:
import pandas as pd

In [ ]:
df_logs = pd.DataFrame(columns=["run_id", "code", "description"])

## 2. Execute table-specific Silver

Execute table-specific Silver, by building the target notebook name dynamically.

Log code meanings:

* **02**: success
* **01**: failure

In [ ]:
v_message = ""
v_notebook_name = f"04_notebook_{in_parameter_table_name}_silver_layer"
try:
    if in_parameter_is_fact_table == True:
        out_parameter_count_processed = mssparkutils.notebook.run(v_notebook_name, timeout_seconds = 30, arguments = {
            "in_parameter_run_id": in_parameter_run_id,
            "in_parameter_process_date": in_parameter_process_date,
            "in_parameter_year": in_parameter_year
        })
    else:
        out_parameter_count_processed = mssparkutils.notebook.run(v_notebook_name, timeout_seconds = 30, arguments = {
            "in_parameter_run_id": in_parameter_run_id,
            "in_parameter_process_date": in_parameter_process_date
        })
    df_logs.loc[0] = [in_parameter_run_id, "02", f"{v_notebook_name}"]
except Exception as e:
    v_message = str(e)
    df_logs.loc[0] = [in_parameter_run_id, "01", f"{v_notebook_name} - {v_message}"]

## 3. Persist logs

Convert the log entries into a Spark DataFrame and append them to control.logs for centralized run monitoring.

In [ ]:
df_logs = spark.createDataFrame(df_logs)
df_logs = df_logs.withColumn("run_id", col("run_id").cast(IntegerType()))
df_logs.write.format("delta").mode("append").saveAsTable("control.logs")

Convert the log entries into a Spark DataFrame and append them to control.logs for centralized run monitoring.

In [ ]:
mssparkutils.notebook.exit(out_parameter_count_processed)